# Predict price diff for BTC_ETC

In [1]:
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier

import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('../datasets/5min/BTC_ETC.csv', index_col=0)

In [3]:
df.head()

,open,high,low,close,volume
datetime,,,,,
1473608100,0.002170,0.00217,0.002170,0.002170,0.013235
1473608400,0.002170,0.00218,0.002170,0.002175,10.628815
1473608700,0.002176,0.00218,0.002175,0.002175,0.535791
1473609000,0.002180,0.00218,0.002175,0.002178,0.420742
1473609300,0.002175,0.00218,0.002175,0.002175,0.148593


In [4]:
max_lag = 12
for lag in range(1, max_lag+1):
    df['close-' + str(lag)] = df['close'] - df.shift(lag)['close']

In [5]:
df.head(13)

,open,high,low,close,volume,close-1,close-2,close-3,close-4,close-5,close-6,close-7,close-8,close-9,close-10,close-11,close-12
datetime,,,,,,,,,,,,,,,,,
1473608100,0.002170,0.002170,0.002170,0.002170,0.013235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1473608400,0.002170,0.002180,0.002170,0.002175,10.628815,4.990000e-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1473608700,0.002176,0.002180,0.002175,0.002175,0.535791,0.000000e+00,4.990000e-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1473609000,0.002180,0.002180,0.002175,0.002178,0.420742,2.970000e-06,2.970000e-06,7.960000e-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1473609300,0.002175,0.002180,0.002175,0.002175,0.148593,-2.720000e-06,2.500000e-07,2.500000e-07,5.240000e-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1473609600,0.002175,0.002178,0.002175,0.002178,0.026276,2.780000e-06,6.000000e-08,3.030000e-06,3.030000e-06,0.000008,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1473609900,0.002180,0.002183,0.002180,0.002183,4.623415,4.790000e-06,7.570000e-06,4.850000e-06,7.820000e-06,0.000008,0.000013,NaN,NaN,NaN,NaN,NaN,NaN
1473610200,0.002180,0.002180,0.002170,0.002178,3.489886,-4.620000e-06,1.700000e-07,2.950000e-06,2.300000e-07,0.000003,0.000003,0.000008,NaN,NaN,NaN,NaN,NaN
1473610500,0.002179,0.002179,0.002170,0.002170,4.094033,-8.070000e-06,-1.269000e-05,-7.900000e-06,-5.120000e-06,-0.000008,-0.000005,-0.000005,1.200000e-07,NaN,NaN,NaN,NaN


In [29]:
df['up'] = df['close'] < df.shift(-1)['close']
df['close_next'] = df.shift(-1)['close']
df.head()

,open,high,low,close,volume,close-1,close-2,close-3,close-4,close-5,close-6,close-7,close-8,close-9,close-10,close-11,close-12,up,close_next
datetime,,,,,,,,,,,,,,,,,,,
1473611700,0.002174,0.002174,0.002174,0.002174,0.020336,1.700000e-07,1.206000e-05,0.000009,0.000004,-0.000004,-0.000009,-0.000004,-0.000001,-4.010000e-06,-0.000001,-0.000001,0.000004,True,0.002176
1473612000,0.002174,0.002176,0.002174,0.002176,0.339290,2.200000e-06,2.370000e-06,0.000014,0.000011,0.000006,-0.000002,-0.000007,-0.000002,9.100000e-07,-0.000002,0.000001,0.000001,True,0.002180
1473612300,0.002176,0.002183,0.002174,0.002180,2.238676,3.900000e-06,6.100000e-06,0.000006,0.000018,0.000015,0.000010,0.000002,-0.000003,2.030000e-06,0.000005,0.000002,0.000005,True,0.002183
1473612600,0.002180,0.002183,0.002180,0.002183,3.898806,2.760000e-06,6.660000e-06,0.000009,0.000009,0.000021,0.000018,0.000013,0.000005,0.000000e+00,0.000005,0.000008,0.000005,False,0.002180
1473612900,0.002183,0.002183,0.002180,0.002180,2.556777,-2.550000e-06,2.100000e-07,0.000004,0.000006,0.000006,0.000018,0.000015,0.000010,2.070000e-06,-0.000003,0.000002,0.000005,True,0.002181


In [10]:
df['up'].value_counts()

False    57353
True     47767
Name: up, dtype: int64

In [11]:
df.dropna(inplace=True)
X = df[['close-1', 'close-2', 'close-3', 'close-4', 'close-5', 'close-6', 'close-6', 'close-7', 'close-8', 'close-9', 'close-10', 'close-11', 'close-12', 'volume']].values
y = df['up'].astype(int).values

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=0)


In [12]:
rf = RandomForestClassifier(n_jobs=4, random_state=0, verbose=2, n_estimators=20)
rf.fit(X_train, y_train)

building tree 1 of 20building tree 2 of 20building tree 3 of 20building tree 4 of 20



building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20


[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    3.2s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=4,
            oob_score=False, random_state=0, verbose=2, warm_start=False)

In [13]:
rf.score(X_test, y_test)

[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    0.1s finished


0.54990010465226902

In [14]:
sgd = SGDClassifier(loss="hinge", penalty="l2", verbose=2)
sgd.fit(X_train, y_train)

-- Epoch 1
Norm: 2.65, NNZs: 14, Bias: 0.303631, T: 84086, Avg. loss: 2.944874
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 1.40, NNZs: 14, Bias: -0.003257, T: 168172, Avg. loss: 1.182097
Total training time: 0.03 seconds.
-- Epoch 3
Norm: 1.17, NNZs: 14, Bias: -0.533167, T: 252258, Avg. loss: 1.071968
Total training time: 0.05 seconds.
-- Epoch 4
Norm: 0.99, NNZs: 14, Bias: -0.721656, T: 336344, Avg. loss: 1.031029
Total training time: 0.06 seconds.
-- Epoch 5
Norm: 0.95, NNZs: 14, Bias: -0.512667, T: 420430, Avg. loss: 1.005704
Total training time: 0.07 seconds.


/home/vampire/PycharmProjects/mikasa_ed/venv/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=2, warm_start=False)

In [15]:
sgd.score(X_test, y_test)

0.55018551993150033

In [74]:
from sklearn import svm
svc = svm.SVC(verbose=2)
svc.fit(X_train, y_train)

[LibSVM]

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=2)

In [76]:
svc.score(X_test, y_test)

0.55984320888990369

In [16]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(verbose=2)
model.fit(X_train, y_train)
model.score(X_test, y_test)

[LibLinear]

0.54671296736751973

In [17]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.54761678241841882

In [18]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.53948244696032732

In [19]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.51688707068785078

In [20]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.57287603463038717

In [24]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(max_depth=4, verbose=1)
model.fit(X_train, y_train)
model.score(X_test, y_test)

      Iter       Train Loss   Remaining Time 
         1           1.3735           19.39s
         2           1.3700           19.19s
         3           1.3667           18.73s
         4           1.3640           18.65s
         5           1.3619           18.34s
         6           1.3599           18.26s
         7           1.3582           18.16s
         8           1.3568           17.99s
         9           1.3552           17.84s
        10           1.3541           17.57s
        20           1.3450           15.22s
        30           1.3398           13.15s
        40           1.3364           11.17s
        50           1.3331            9.24s
        60           1.3307            7.34s
        70           1.3287            5.47s
        80           1.3266            3.64s
        90           1.3248            1.83s
       100           1.3233            0.00s


0.57611074112834171

In [27]:
import numpy as np

np.random.seed(0)

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense


def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(64, input_dim=14, kernel_initializer='normal', activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [28]:
estimator = KerasClassifier(build_fn=create_baseline, nb_epoch=100, batch_size=10, verbose=1)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
results = cross_val_score(estimator, X, y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/10
94596/94596 [==============================] - 11s - loss: 0.6785 - acc: 0.5646    
Epoch 2/10
94596/94596 [==============================] - 12s - loss: 0.6755 - acc: 0.5691    
Epoch 3/10
94596/94596 [==============================] - 12s - loss: 0.6749 - acc: 0.5695    
Epoch 4/10
94596/94596 [==============================] - 12s - loss: 0.6747 - acc: 0.5701    
Epoch 5/10
94596/94596 [==============================] - 12s - loss: 0.6741 - acc: 0.5709    
Epoch 6/10
94596/94596 [==============================] - 11s - loss: 0.6741 - acc: 0.5726    
Epoch 7/10
94596/94596 [==============================] - 12s - loss: 0.6738 - acc: 0.5713    
Epoch 8/10
94596/94596 [==============================] - 12s - loss: 0.6736 - acc: 0.5721    
Epoch 9/10
94596/94596 [==============================] - 11s - loss: 0.6734 - acc: 0.5723    
Epoch 10/10
94597/94597 [==============================] - 12s - loss: 0.6783 - acc: 0.5633    
Epoch 2/10
94597/94597 [=========================

94598/94598 [==============================] - 12s - loss: 0.6759 - acc: 0.5663    
Epoch 3/10
94598/94598 [==============================] - 13s - loss: 0.6753 - acc: 0.5665    
Epoch 4/10
94598/94598 [==============================] - 13s - loss: 0.6748 - acc: 0.5672    
Epoch 5/10
94598/94598 [==============================] - 12s - loss: 0.6747 - acc: 0.5689    
Epoch 6/10
94598/94598 [==============================] - 12s - loss: 0.6743 - acc: 0.5695    
Epoch 7/10
94598/94598 [==============================] - 12s - loss: 0.6738 - acc: 0.5715    
Epoch 8/10
94598/94598 [==============================] - 12s - loss: 0.6738 - acc: 0.5702    
Epoch 9/10
94598/94598 [==============================] - 11s - loss: 0.6735 - acc: 0.5716    
Epoch 10/10
94598/94598 [==============================] - 13s - loss: 0.6781 - acc: 0.5642    
Epoch 2/10
94598/94598 [==============================] - 12s - loss: 0.6756 - acc: 0.5686    
Epoch 3/10
94598/94598 [==============================] - 12